In [3]:
import numpy as np
import os
import poly_yolo as yolo #or "import poly_yolo_lite as yolo" for the lite version
#import poly_yolo_lite as yolo

import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import keras
from keras.models import model_from_json, load_model

In [4]:
#Set the learning phase to Test since the model is already trained.
keras.backend.set_learning_phase(0)

#load pretrained model
#if you want to detect more objects, lower the score and vice versa
trained_model = yolo.YOLO(model_path='models/poly_yolo.h5', iou=0.5, score=0.3)

models/poly_yolo.h5 model, anchors, and classes loaded.


In [5]:
print(trained_model.yolo_model.layers[0].input)
print(trained_model.yolo_model.layers[-1].output)

Tensor("input_2:0", shape=(?, ?, ?, 3), dtype=float32)
Tensor("conv2d_138/BiasAdd:0", shape=(?, ?, ?, 810), dtype=float32)


In [6]:
#save checkpoint
saver = tf.train.Saver()
tf_session = keras.backend.get_session()
input_graph_def = tf_session.graph.as_graph_def()
save_path = saver.save(tf_session, './checkpoint.ckpt')
tf.train.write_graph(input_graph_def,
                     './', 'poly_yolo_pynq.pb', as_text=False)

'./poly_yolo_pynq.pb'

In [7]:
#check the node names 
nodes_names = [node.name for node in 
               tf.get_default_graph().as_graph_def().node]
print(nodes_names)

['input_1', 'conv2d_1/random_uniform/shape', 'conv2d_1/random_uniform/min', 'conv2d_1/random_uniform/max', 'conv2d_1/random_uniform/RandomUniform', 'conv2d_1/random_uniform/sub', 'conv2d_1/random_uniform/mul', 'conv2d_1/random_uniform', 'conv2d_1/kernel', 'conv2d_1/kernel/Assign', 'conv2d_1/kernel/read', 'conv2d_1/weight_regularizer/Square', 'conv2d_1/weight_regularizer/Const', 'conv2d_1/weight_regularizer/Sum', 'conv2d_1/weight_regularizer/mul/x', 'conv2d_1/weight_regularizer/mul', 'conv2d_1/weight_regularizer/add/x', 'conv2d_1/weight_regularizer/add', 'conv2d_1/convolution', 'batch_normalization_1/Const', 'batch_normalization_1/gamma', 'batch_normalization_1/gamma/Assign', 'batch_normalization_1/gamma/read', 'batch_normalization_1/Const_1', 'batch_normalization_1/beta', 'batch_normalization_1/beta/Assign', 'batch_normalization_1/beta/read', 'batch_normalization_1/Const_2', 'batch_normalization_1/moving_mean', 'batch_normalization_1/moving_mean/Assign', 'batch_normalization_1/moving_m

In [8]:
#!freeze_graph --help

In [9]:
#Freeze Tensorflow graph: lastnode: concat_9; lastlayer: conv2d_69/BiasAdd
!freeze_graph \
    --input_meta_graph checkpoint.ckpt.meta \
    --input_checkpoint checkpoint.ckpt \
    --output_graph frozen.pb \
    --output_node_names conv2d_69/BiasAdd \
    --variable_names_blacklist batch_normalization_1/gamma/read, batch_normalization_1/beta/read \
    --input_binary true

Loaded meta graph file 'checkpoint.ckpt.meta
Instructions for updating:
Use standard file APIs to check for files with this prefix.
W1012 00:24:29.624882 139963591964480 deprecation.py:323] From /opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib/python3.6/site-packages/tensorflow_core/python/tools/freeze_graph.py:127: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
2020-10-12 00:24:29.658275: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2899885000 Hz
2020-10-12 00:24:29.659324: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55688677c450 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-10-12 00:24:29.659351: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
INFO:tensorflow:Restoring par

In [10]:
#confirm available input and output node names
!vai_q_tensorflow inspect --input_frozen_graph=frozen.pb

Op types used: 926 Const, 396 Identity, 189 Reshape, 95 LeakyRelu, 84 Shape, 81 Pack, 69 Conv2D, 68 FusedBatchNormV3, 54 Unpack, 54 Transpose, 54 MatMul, 30 Mul, 30 StridedSlice, 30 AddV2, 27 Mean, 27 Sigmoid, 5 Pad, 3 ResizeBilinear, 1 Placeholder, 1 BiasAdd

Found 1 possible inputs: (name=input_1, type=float(1), shape=[?,?,?,3]) 
Found 1 possible outputs: (name=conv2d_69/BiasAdd, op=BiasAdd) 


#Prepare calibration data and function

In [11]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

dir_imgs_name = '../cityscapes/leftImg8bit_demoVideo/leftImg8bit/demoVideo/stuttgart_01/'  #path_where_are_images_to_clasification
list_of_imgs = [root+"/"+name for root, dirs, files in os.walk(dir_imgs_name) for name in files]    
list_of_imgs.sort()
num_imgs = 32

# get the 1st image with right shape
img = load_img(list_of_imgs[0])  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (1024, 2048, 3)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 1024, 2048, 3)
dataset_calib = x
# fill in the following images
imgs          = 0
for im in range (1, num_imgs):#len(list_of_imgs)):
    imgs    += 1
    img = load_img(list_of_imgs[im])  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (1024, 2048, 3)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 1024, 2048, 3)
    dataset_calib = np.append(dataset_calib, x, axis=0)
print('Calibration data: {}, {}, {}'.format(dataset_calib.shape, dataset_calib.ndim, dataset_calib.dtype))

np.savez('./calib_data.npz', data = dataset_calib[:num_imgs])

del(dataset_calib)

Calibration data: (32, 1024, 2048, 3), 4, float32


In [12]:
%%writefile input_func.py
import numpy as np

data = np.load('calib_data.npz')['data']

batch_size=1

def calib_input(iter):

    calib_data = data[iter*batch_size:(iter+1)*batch_size]

    return {'input_1': calib_data}

Overwriting input_func.py


In [13]:
#!vai_q_tensorflow quantize --help

In [14]:
!vai_q_tensorflow quantize \
    --input_frozen_graph frozen.pb \
    --input_fn input_func.calib_input \
    --output_dir quantized \
    --input_nodes input_1 \
    --output_nodes conv2d_69/BiasAdd \
    --input_shapes ?,1024,2048,3 \
    --skip_check 0 \
    --simulate_dpu 1 \
    --calib_iter 1

INFO: Checking Float Graph...
INFO: Float Graph Check Done.
2020-10-12 00:24:50.371686: W tensorflow/contrib/decent_q/utils/quantize_utils.cc:722] Convert alpha of leaky_relu nodeleaky_re_lu_1/LeakyRelu from 0.1 to 26/256 to simulate DPU.
2020-10-12 00:24:50.371749: W tensorflow/contrib/decent_q/utils/quantize_utils.cc:722] Convert alpha of leaky_relu nodeleaky_re_lu_2/LeakyRelu from 0.1 to 26/256 to simulate DPU.
2020-10-12 00:24:50.371788: W tensorflow/contrib/decent_q/utils/quantize_utils.cc:722] Convert alpha of leaky_relu nodeleaky_re_lu_3/LeakyRelu from 0.1 to 26/256 to simulate DPU.
2020-10-12 00:24:50.371827: W tensorflow/contrib/decent_q/utils/quantize_utils.cc:722] Convert alpha of leaky_relu nodeleaky_re_lu_4/LeakyRelu from 0.1 to 26/256 to simulate DPU.
2020-10-12 00:24:50.371881: W tensorflow/contrib/decent_q/utils/quantize_utils.cc:722] Convert alpha of leaky_relu nodeleaky_re_lu_5/LeakyRelu from 0.1 to 26/256 to simulate DPU.
2020-10-12 00:24:50.371941: W tensorflow/cont

2020-10-12 00:24:51.230565: W tensorflow/contrib/decent_q/utils/graph_quantizer.cc:347] Quantize convfc + bias + leakyrelu: leaky_re_lu_1/LeakyRelu(LeakyRelu) <-- batch_normalization_1/FusedBatchNormV3_1/add(Add) <-- conv2d_1/convolution(Conv2D)
2020-10-12 00:24:51.230598: W tensorflow/contrib/decent_q/utils/graph_quantizer.cc:347] Quantize convfc + bias + leakyrelu: leaky_re_lu_2/LeakyRelu(LeakyRelu) <-- batch_normalization_2/FusedBatchNormV3_1/add(Add) <-- conv2d_2/convolution(Conv2D)
2020-10-12 00:24:51.230615: W tensorflow/contrib/decent_q/utils/graph_quantizer.cc:347] Quantize convfc + bias + leakyrelu: leaky_re_lu_3/LeakyRelu(LeakyRelu) <-- batch_normalization_3/FusedBatchNormV3_1/add(Add) <-- conv2d_3/convolution(Conv2D)
2020-10-12 00:24:51.230631: W tensorflow/contrib/decent_q/utils/graph_quantizer.cc:347] Quantize convfc + bias + leakyrelu: leaky_re_lu_4/LeakyRelu(LeakyRelu) <-- batch_normalization_4/FusedBatchNormV3_1/add(Add) <-- conv2d_4/convolution(Conv2D)
2020-10-12 00:24

INFO: Calibrating for 1 iterations...
100% (1 of 1) |##########################| Elapsed Time: 0:01:08 Time:  0:01:08
INFO: Calibration Done.
INFO: Generating Deploy Model...
[DEPLOY WARNING] Node multiply_1/mul(Type: Mul) is not quantized and cannot be deployed to DPU,because it has unquantized input node: dense_2/Sigmoid. Please deploy it on CPU.
[DEPLOY WARNING] Node add_1/add(Type: Add) is not quantized and cannot be deployed to DPU,because it has unquantized input node: multiply_1/mul. Please deploy it on CPU.
[DEPLOY WARNING] Node zero_padding2d_2/Pad(Type: Pad) is not quantized and cannot be deployed to DPU,because it has unquantized input node: add_1/add. Please deploy it on CPU.
[DEPLOY WARNING] Node multiply_2/mul(Type: Mul) is not quantized and cannot be deployed to DPU,because it has unquantized input node: dense_4/Sigmoid. Please deploy it on CPU.
[DEPLOY WARNING] Node add_2/add(Type: Add) is not quantized and cannot be deployed to DPU,because it has unquantized input node

INFO: Deploy Model Generated.
********************* Quantization Summary *********************      
INFO: Output:       
  quantize_eval_model: quantized/quantize_eval_model.pb       
  deploy_model: quantized/deploy_model.pb


In [17]:
!vai_c_tensorflow \
    --frozen_pb quantized/deploy_model.pb \
    --arch ../../Ultra96.json \
    --output_dir . \
    --net_name poly_yolo_dnn_quantized

**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[VAI_C][Error] Unrecognized tensorflow op [Mul].
[VAI_C][Error] Parsing tensorflow model failed.
